In [ ]:
#demonstrates how someone can take saved model and use it without retraining.
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

#  STEP 1: RE-DEFINE THE MODEL ARCHITECTURE
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv_stack = nn.Sequential(
            nn.Conv2d(1, 6, 5),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(6, 16, 5),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(16 * 4 * 4, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, 10) 
        )

    def forward(self, x):
        x = self.conv_stack(x)
        x = torch.flatten(x, 1) # Flatten all dimensions except batch
        x = self.classifier(x)
        return x

#  STEP 2: DEFINE VARIABLES & LOAD MODEL
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
PATH = './my_cnn_model.pth'

loaded_net = Net()
loaded_net.load_state_dict(torch.load(PATH, weights_only=True))
loaded_net.to(device)

print("Model loaded successfully from disk and moved to GPU.")

#  STEP 3: LOAD TEST DATA (New Part)
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,), (0.5,))])

testset = torchvision.datasets.FashionMNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False)

print("Test data loaded.")

#  STEP 4: VERIFY THE LOADED MODEL'S ACCURACY (New Part)
correct = 0
total = 0

# Set model to evaluation mode (disables dropout, etc.)
loaded_net.eval()

with torch.no_grad(): # We don't need to calculate gradients
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        
        outputs = loaded_net(images)
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy of the LOADED model: {accuracy:.2f} %')

C:\Users\ASUS\AppData\Local\Temp\ipykernel_22576\2925073278.py:43: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_net.load_state_dict(torch.load(PATH))


Model loaded successfully from disk and moved to GPU.
Test data loaded.
Accuracy of the LOADED model: 88.02 %
